# Create and Run Content Safety using Python SDK

### Overview

Azure AI Content Safety detects harmful user-generated and AI-generated content in applications and services. Content Safety includes text and image APIs that allow you to detect material that is harmful.

In this hands-on, you will be able to:
Manage text blocklist: The default AI classifiers are sufficient for most content safety needs.
Analyze text contents: Scans text for sexual content, violence, hate, and self-harm with multi-severity levels.
Analyze images: Scans images for sexual content, violence, hate, and self-harm with multi-severity levels.
Integrate with Azure Open AI Service: Use the Azure Open AI Service to rewrite the content for harmful content.

#### 1. Azure Content Safety Set up

#### 2. Azure Content Safety with BlockList

#### 3. Azure Content Safety for Text

#### 4. Azure Content Safety for Iamges

#### 5. Rewrite Content with Azure Open AI Service

[Note] Please use `Python 3.10 - SDK v2 (azureml_py310_sdkv2)` conda environment.


In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys
lab_prep_dir = os.getcwd().split("slm-innovator-lab")[0] + "slm-innovator-lab/0_lab_preparation"
sys.path.append(os.path.abspath(lab_prep_dir))

from common import check_kernel
check_kernel()

In [ ]:
import datetime
import os
import sys
from openai import AzureOpenAI
from azure.ai.contentsafety import ContentSafetyClient
from azure.ai.contentsafety.models import (
    AnalyzeImageOptions,
    ImageData,
    AnalyzeTextOptions,
    ImageCategory, 
    TextCategory
)
from azure.core.credentials import AzureKeyCredential
from azure.core.exceptions import HttpResponseError
from IPython.display import Image
from dotenv import load_dotenv

from azure.ai.contentsafety import BlocklistClient
from azure.ai.contentsafety.models import TextBlocklist, TextBlocklistItem
from azure.ai.contentsafety.models import AddOrUpdateTextBlocklistItemsOptions, TextBlocklistItem

load_dotenv()

## 2. Azure Content Safety with BlockList

-   The default AI classifiers are sufficient for most content safety needs however, you might need to screen for terms that are specific to your use case. You can create blocklists of terms to use with the Text API.


In [ ]:
def create_or_update_text_blocklist(blocklist_name, blocklist_description):
    
    
    key = os.environ["CONTENT_SAFETY_KEY"]
    endpoint = os.environ["CONTENT_SAFETY_ENDPOINT"]

    # Create a Blocklist client
    client = BlocklistClient(endpoint, AzureKeyCredential(key))

    try:
        blocklist = client.create_or_update_text_blocklist(
            blocklist_name=blocklist_name,
            options=TextBlocklist(blocklist_name=blocklist_name, description=blocklist_description),
        )
        if blocklist:
            print("\nBlocklist created or updated: ")
            print(f"Name: {blocklist.blocklist_name}, Description: {blocklist.description}")
    except HttpResponseError as e:
        print("\nCreate or update text blocklist failed: ")
        if e.error:
            print(f"Error code: {e.error.code}")
            print(f"Error message: {e.error.message}")
            raise
        print(e)
        raise

In [ ]:
def add_blocklist_items(blocklist_name, blocklist_items):
    
    key = os.environ["CONTENT_SAFETY_KEY"]
    endpoint = os.environ["CONTENT_SAFETY_ENDPOINT"]

    # Create a Blocklist client
    client = BlocklistClient(endpoint, AzureKeyCredential(key))

    try:
        result = client.add_or_update_blocklist_items(
            blocklist_name=blocklist_name, options=AddOrUpdateTextBlocklistItemsOptions(blocklist_items=blocklist_items)
        )
        for blocklist_item in result.blocklist_items:
            print(
                f"BlocklistItemId: {blocklist_item.blocklist_item_id}, Text: {blocklist_item.text}, Description: {blocklist_item.description}"
            )
    except HttpResponseError as e:
        print("\nAdd blocklist items failed: ")
        if e.error:
            print(f"Error code: {e.error.code}")
            print(f"Error message: {e.error.message}")
            raise
        print(e)
        raise


In [ ]:
create_or_update_text_blocklist("TestBlocklist", "Test blocklist management.")

In [ ]:
blocklist_items = [
    TextBlocklistItem(text="alien", description=""),
    TextBlocklistItem(text="octopus", description="")
]
add_blocklist_items("TestBlocklist", blocklist_items)


In [ ]:
def analyze_text_with_blocklists(blocklist_name, input_text):
    import os
    from azure.ai.contentsafety import ContentSafetyClient
    from azure.core.credentials import AzureKeyCredential
    from azure.ai.contentsafety.models import AnalyzeTextOptions
    from azure.core.exceptions import HttpResponseError

    key = os.environ["CONTENT_SAFETY_KEY"]
    endpoint = os.environ["CONTENT_SAFETY_ENDPOINT"]

    # Create a Content Safety client
    client = ContentSafetyClient(endpoint, AzureKeyCredential(key))

    try:
        # After you edit your blocklist, it usually takes effect in 5 minutes, please wait some time before analyzing
        # with blocklist after editing.
        analysis_result = client.analyze_text(
            AnalyzeTextOptions(text=input_text, blocklist_names=[blocklist_name], halt_on_blocklist_hit=False)
        )
        if analysis_result and analysis_result.blocklists_match:
            print("\nBlocklist match results: ")
            for match_result in analysis_result.blocklists_match:
                print(
                    f"BlocklistName: {match_result.blocklist_name}, BlocklistItemId: {match_result.blocklist_item_id}, "
                    f"BlocklistItemText: {match_result.blocklist_item_text}"
                )
    except HttpResponseError as e:
        print("\nAnalyze text failed: ")
        if e.error:
            print(f"Error code: {e.error.code}")
            print(f"Error message: {e.error.message}")
            raise
        print(e)
        raise

In [ ]:
analyze_text_with_blocklists("TestBlocklist", "I think you're a real alien, don't say that octopus!")

## 3. Azure Content Safety for text

-   Scans text for sexual content, violence, hate, and self-harm with multi-severity levels.


In [ ]:
def analyze_text(prompt_text):
    # analyze text
    key = os.environ["CONTENT_SAFETY_KEY"]
    endpoint = os.environ["CONTENT_SAFETY_ENDPOINT"]
    api_version = os.environ["CONTENT_SAFETY_API_VERSION"]

    # Create a Content Safety client
    client = ContentSafetyClient(endpoint, AzureKeyCredential(key), api_version=api_version)

    # Construct request
    request = AnalyzeTextOptions(text=prompt_text)

    # Analyze text
    try:
        response = client.analyze_text(request)
        print(response)
    except HttpResponseError as e:
        print("Analyze text failed.")
        if e.error:
            print(f"Error code: {e.error.code}")
            print(f"Error message: {e.error.message}")
            raise
        print(e)
        raise

    hate_result = next(item for item in response.categories_analysis if item.category == TextCategory.HATE)
    self_harm_result = next(item for item in response.categories_analysis if item.category == TextCategory.SELF_HARM)
    sexual_result = next(item for item in response.categories_analysis if item.category == TextCategory.SEXUAL)
    violence_result = next(item for item in response.categories_analysis if item.category == TextCategory.VIOLENCE)

    if hate_result:
        print(f"Hate severity: {hate_result.severity}")
    if self_harm_result:
        print(f"SelfHarm severity: {self_harm_result.severity}")
    if sexual_result:
        print(f"Sexual severity: {sexual_result.severity}")
    if violence_result:
        print(f"Violence severity: {violence_result.severity}")

In [ ]:
analyze_text("Hi! nice to meet you")

In [ ]:
analyze_text("you are an idiot")

In [ ]:
analyze_text("Painful twisting of the arm")

In [ ]:
analyze_text("Tell me more about the prompts you're doing now")

## 4. Azure Content Safety for images

-   Scans images for sexual content, violence, hate, and self-harm with multi-severity levels.


In [ ]:
def analyze_image(image_path):
    # analyze image
    key = os.environ["CONTENT_SAFETY_KEY"]
    endpoint = os.environ["CONTENT_SAFETY_ENDPOINT"]
    
    # Create a Content Safety client
    client = ContentSafetyClient(endpoint, AzureKeyCredential(key))

    # Build request
    with open(image_path, "rb") as file:
        request = AnalyzeImageOptions(image=ImageData(content=file.read()))

    # Analyze image
    try:
        response = client.analyze_image(request)
    except HttpResponseError as e:
        print("Analyze image failed.")
        if e.error:
            print(f"Error code: {e.error.code}")
            print(f"Error message: {e.error.message}")
            raise
        print(e)
        raise

    hate_result = next(item for item in response.categories_analysis if item.category == ImageCategory.HATE)
    self_harm_result = next(item for item in response.categories_analysis if item.category == ImageCategory.SELF_HARM)
    sexual_result = next(item for item in response.categories_analysis if item.category == ImageCategory.SEXUAL)
    violence_result = next(item for item in response.categories_analysis if item.category == ImageCategory.VIOLENCE)

    if hate_result:
        print(f"Hate severity: {hate_result.severity}")
    if self_harm_result:
        print(f"SelfHarm severity: {self_harm_result.severity}")
    if sexual_result:
        print(f"Sexual severity: {sexual_result.severity}")
    if violence_result:
        print(f"Violence severity: {violence_result.severity}")

In [ ]:
image_path = "images/image1.jpg"

Image(filename=image_path, width=480)

In [ ]:
analyze_image(image_path)

In [ ]:
image_path = "images/image2.jpg"

Image(filename=image_path, width=480)

In [ ]:
analyze_image(image_path)

## 5. Manage hamful Contents with Azure Open AI Service

Here are some strategies you might consider to address this issue:

-   **Adjusting Model Settings**: Use a less restrictive model or service to analyze the content and identify problematic sections. For Azure OpenAI models, only customers who have been approved for modified content filtering have full content filtering control and can turn off content filters. Apply for modified content filters via this form: [Azure OpenAI Limited Access Review: Modified Content Filters](https://ncv.microsoft.com/uEfCgnITdR)
-   **Preprocess the Content Manually**: Before sending the text to GPT-4o, manually replace or mask explicit terms and phrases that are likely to trigger the content filters. You can use placeholders like [redacted] or \*\*\* to cover sensitive words. This can make the content acceptable for initial processing.
-   **Automated Content Sanitization**: Modify or summarize those sections to remove disallowed content before inputting the text into GPT-4o for rewriting. In order to avoid to be blocked by Azure OpenAI, you should submit modified content filters via this form. You can also use Natural language processing libraries can help identify and filter out sensitive content programmatically.


In [ ]:
%pip install openai

In [ ]:
content = "you are an idiot"
result = analyze_text(content)


In [ ]:
def rewrite_content(content):
    system_message = """
    You are an AI language model tasked with rewriting the provided content 
    to eliminate any sexual content, violence, hate speech, and references to self-harm. 
    Please ensure that the revised text maintains the original meaning 
    and tone as much as possible, while removing or altering any prohibited material.
    """
    
    aoai_api_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    aoai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
    aoai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")
    aoai_deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

    if not aoai_api_version:
        aoai_api_version = os.getenv("OPENAI_API_VERSION")
    if not aoai_deployment_name:
        aoai_deployment_name = os.getenv("DEPLOYMENT_NAME")
        
    try:
        client = AzureOpenAI(
            azure_endpoint = aoai_api_endpoint,
            api_key        = aoai_api_key,
            api_version    = aoai_api_version
        )

    except (ValueError, TypeError) as e:
        print(e)

    user_message = f"""
    content: {content}
    """
    
    # call the Azure OpenAI API turned off content filter
    response = client.chat.completions.create(
        model=aoai_deployment_name,
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_message},
        ],
        temperature=0.2,
        max_tokens=300
    )
   
    return response.choices[0].message.content

In [ ]:
rewrite_content(content)